In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 530.4 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.9 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


#### Load Dataset

In [2]:
import pandas as pd

def load_minimal_fakenewsnet(fake_path, real_path):
    fake_df = pd.read_csv(fake_path, header=0)
    real_df = pd.read_csv(real_path, header=0)
    
    fake_df['label'] = 0
    real_df['label'] = 1
    
    return pd.concat([fake_df, real_df], ignore_index=True)


In [3]:
df = load_minimal_fakenewsnet('dataset/politifact_fake.csv', 'dataset/politifact_real.csv')
df.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,0


In [4]:
def extract_tweets(tweet_str):
    if isinstance(tweet_str, str):
        return tweet_str.strip().split()
    return []


In [5]:
df['tweet_ids'] = df['tweet_ids'].apply(extract_tweets)
df.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,"[937349434668498944, 937379378006282240, 93738...",0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,"[972666281441878016, 972678396575559680, 97282...",0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,"[929405740732870656, 929439450400264192, 92943...",0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,"[886941526458347521, 887011300278194176, 88702...",0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,"[915205698212040704, 915242076681506816, 91524...",0


#### Preprocessing title

In [9]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import spacy
import pandas as pd

# Load English model
nlp = spacy.load("en_core_web_sm")

def preprocess_title_spacy(title):
    """
    Preprocess news titles using spaCy:
    - Lowercase
    - Tokenize
    - Lemmatize
    - Remove stopwords, punctuations, numbers
    - Keep words longer than 2 characters
    """
    if not isinstance(title, str):
        return ""
    
    doc = nlp(title.lower())
    
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop
        and not token.is_punct
        and token.is_alpha
        and len(token.lemma_) > 2
    ]
    
    return ' '.join(tokens)


In [11]:
df["preprocessed_title"] = df["title"].apply(preprocess_title_spacy)
print(df[['title', 'preprocessed_title']])

                                                  title  \
0     BREAKING: First NFL Team Declares Bankruptcy O...   
1     Court Orders Obama To Pay $400 Million In Rest...   
2     UPDATE: Second Roy Moore Accuser Works For Mic...   
3            Oscar Pistorius Attempts To Commit Suicide   
4           Trump Votes For Death Penalty For Being Gay   
...                                                 ...   
1051  Flake: “Religious tests should have no place i...   
1052                           Change We Can Believe In   
1053  deputy director of national health statistics ...   
1054  Romneys ProLife Conversion Myth or Reality Jun...   
1055                             Interest Group Ratings   

                                     preprocessed_title  
0       breaking nfl team declare bankruptcy kneel thug  
1             court order obama pay million restitution  
2     update second roy moore accuser work michelle ...  
3                oscar pistorius attempt commit suicide  
4

#### Create Nodes

In [22]:
news_nodes = list(df['id'])
tweet_nodes = list({t for lst in df['tweet_ids'] for t in lst})
print("No. of news nodes:", len(news_nodes))
print("No. of tweet nodes:", len(tweet_nodes))
print("Total No. of nodes:", len(news_nodes) + len(tweet_nodes))

No. of news nodes: 1056
No. of tweet nodes: 558937
Total No. of nodes: 559993


#### Create mappings (for indexing):

In [23]:
news2idx = {nid: i for i, nid in enumerate(news_nodes)}
tweet2idx = {tid: i + len(news_nodes) for i, tid in enumerate(tweet_nodes)}

#### Build Edges (Tweet → News)

In [32]:
edges_src = []
edges_dst = []

for _, row in df.iterrows():
    n_id = news2idx[row['id']]
    for t_id in row['tweet_ids']:
        if t_id in tweet2idx:
            edges_src.append(tweet2idx[t_id])  # tweet node index
            edges_dst.append(n_id)             # news node index


In [33]:
# Adding reverse edge for creating the undirected graph
edges_src_rev = edges_dst.copy()
edges_dst_rev = edges_src.copy()

edges_src = edges_src + edges_src_rev
edges_dst = edges_dst + edges_dst_rev

In [34]:
print(len(edges_dst), len(edges_src))

1167032 1167032


In [29]:
!pip install torch torch-geometric

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 547.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 13.4 MB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cac

In [35]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
labels = torch.tensor(df['label'].values, dtype=torch.long)

data = Data(edge_index=edge_index)
data.y = labels
data.num_nodes = len(news_nodes) + len(tweet_nodes)


In [37]:
!pip install numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 631.3 kB/s eta 0:00:0031m? eta -:--:--
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 7.5 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 16.3 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=512)
X_news = vectorizer.fit_transform(df['preprocessed_title']).toarray()

import numpy as np
X_tweets = np.zeros((len(tweet_nodes), 512))  # placeholder
X_all = np.vstack((X_news, X_tweets))

data.x = torch.tensor(X_all, dtype=torch.float)
